In [2]:
import numpy as np

In [5]:
with open('debug_lidar_frames/numfile1.npy', 'rb') as f:

    a = np.load(f)

In [7]:
a.shape

(11909, 3)

In [31]:
dist = [np.linalg.norm(a[i]) for i in range(len(a)) ]
filtered_indexes = [ i for i in range(len(dist)) if( (dist[i] < 60) and (dist[i]> -60))]

In [32]:
lidar = a[filtered_indexes]

In [33]:
max(np.array(dist)[filtered_indexes])

59.99289683397244

In [14]:
z = [a[i][2] for i in range(len(a))]

In [16]:
max(z)

26.853914260864258

In [39]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(lidar)
o3d.io.write_point_cloud("lidar_test_data/point_cloud_data_sample_2.pcd", pcd)

True

In [44]:
import math
class Node:
    """
    data: list of [x,y,z] coordinates
    """
    def __init__(self, data, point_id):
        self.point = {"X": data[0], "Y": data[1], "Z": data[2]}
        self.point_id = point_id
        self.left_node = None
        self.right_node = None

class KdTree_class:
    """
    Implementation of KDtree
    """
    def __init__(self):
        self.root = None
        self.kdtree_display_dict = dict()


    def insert_points(self, pcd_dataframe, display_output=False):
        """
        :param pcd_dataframe: dataframe containing (x,y,z) coordinates data
        :return:
        """
        for row in pcd_dataframe.iterrows():
            x, y, z, point_id = row[1]["X"], row[1]["Y"], row[1]["Z"], row[0]
            point = (x, y, z)
            level = 0
            self.root = self.build_kdtree(self.root, level, point, point_id)
        # If display_output is enabled, then display the contents of Kdtree
        if display_output:
            print("Kdtree Build Complete")
            self.display_kdtree(self.root)
            for pair in self.kdtree_display_dict.items():
                print(f"Depth = {pair[0]}, Points = {pair[1]} ")
        return self.root

    def build_kdtree(self, node, depth, point,point_id):
        """
        :param node: Node class object
        :param depth: level0 -x, level1-y, Level2-z
        :return: root node
        """
        if node is None:
            node = Node(point, point_id)
            return node

        # If current node is empty, then assign the point as root
        current_node = Node(point, point_id)
        # Level should always be within 0-2 range
        depth %= 3
        # levels correspond to (X,Y,Z), check at each level before assigning left/right to the root node
        # self.__dict_key returns X,Y,Z based on the level value. Check function for detailed description
        if node.point[self.__dict_key(depth)] < current_node.point[self.__dict_key(depth)]:
            # If value at level less than current node point, add it as a right node
            node.right_node = self.build_kdtree(node.right_node, depth + 1, point, point_id)
        else:
            # If value at level is more than current node point, add it as a left node
            node.left_node = self.build_kdtree(node.left_node, depth + 1, point, point_id)
        return node

    def search_elements(self, node, search_point, distance_threshold, depth=0, kdtree_search_results=set()):
        """
        :param node: node of kdtree
        :param search_point: (x,y,z) point
        :param distance_threshold: pcd elements near point
        :param depth: level of the kdtree depth
        :param kdtree_search_results: level of the kdtree depth
        :return: ids which can be considered as near points
        """
        depth %= 3
        current_node = node
        if current_node is not None:
            # If current node is within the distance threshold of search_point
            if(((current_node.point["X"] < search_point[0] + distance_threshold) and (current_node.point["X"] > search_point[0] - distance_threshold)) and
               ((current_node.point["Y"] < search_point[1] + distance_threshold) and (current_node.point["Y"] > search_point[1] - distance_threshold)) and
               ((current_node.point["Z"] < search_point[2] + distance_threshold) and (current_node.point["Z"] > search_point[2] - distance_threshold))):
                # Calculate the distance of search point from current node
                point_distance = math.sqrt(math.pow((current_node.point["X"] - search_point[0]), 2) +
                                           math.pow((current_node.point["Y"] - search_point[1]), 2) +
                                           math.pow((current_node.point["Z"] - search_point[2]), 2))
                if point_distance <= distance_threshold:
                    kdtree_search_results.add(current_node.point_id)
                else:
                    pass
            # Iterate recursively
            # if current_node.point[self.__dict_key(depth)] < search_point[depth]:
            if current_node.point[self.__dict_key(depth)] < search_point[depth] + distance_threshold:
                self.search_elements(current_node.right_node, search_point, distance_threshold,
                                     depth+1, kdtree_search_results)

            # else:
            if current_node.point[self.__dict_key(depth)] > search_point[depth] - distance_threshold:
                self.search_elements(current_node.left_node, search_point, distance_threshold,
                                     depth+1, kdtree_search_results)
            return kdtree_search_results

    def display_kdtree(self, node, depth=0):
        """
        updates the self.kdtree_dict with the points are corresponding depth
        :param node: root node
        :param depth: indicates the depth of Kdtree
        """
        current_node = node
        try:
            # If there are values already present, append the list with the point.
            self.kdtree_display_dict[depth].extend([(current_node.point["X"],
                                                    current_node.point["Y"],
                                                    current_node.point["Z"])])
        except KeyError:
            # If there are no values at the level, add value as first point
            self.kdtree_display_dict[depth] = [(current_node.point["X"],
                                                current_node.point["Y"],
                                                current_node.point["Z"])]
        # Run the recursion until a function hits the empty node
        if current_node is not None:
            # Check
            if current_node.left_node is not None:
                left_node = current_node.left_node
                # increment the value of depth
                depth += 1
                # at every node, call the recursive function
                self.display_kdtree(left_node ,depth)

            if current_node.right_node is not None:
                right_node = current_node.right_node
                # increment the value of depth
                depth += 1
                # at every node, call the recursive function
                self.display_kdtree(right_node ,depth)

    @staticmethod
    def __dict_key(number):
        """
        returns XYZ based on number
        :param number: 0,1,2
        :return: X,Y,Z
        """
        try:
            key_dict = {0: "X", 1: "Y", 2: "Z"}
            return str(key_dict[number])
        except KeyError:
            raise Exception(f"Incorrect Level({number}) Assignment.") 

In [45]:
"""
Processing of Point Cloud data
"""
# from Kdtree import KdTree_class
import pandas as pd
import copy
import plotly.graph_objects as go
import open3d as o3d
import sys
# Set the recursion limit to avoid RecursionError exception while building Kdtree
sys.setrecursionlimit(10**8)


class ProcessPointCloud:
    """
    ProcessPointCloud Class
    """
    def __init__(self, pcd_file, nrows_value, display_output_flag):
        self.nrows = nrows_value
        # if pcd_file.endswith(".pcd"):
        #     pcd = o3d.io.read_point_cloud(r"point_cloud_data_sample_2.pcd")
        #     self.pcd_data = pd.DataFrame(pcd.points, columns={"X" ,"Y" ,"Z"})
        #     self.pcd_data = self.pcd_data[0:self.nrows]
        # elif pcd_file.endswith(".xyz"):
        #     self.pcd_data = pd.read_csv("point_cloud_data_sample.xyz", delimiter=" ", nrows=nrows_value)
        if(isinstance(pcd_file, type(np.array([])))):
            self.pcd_data = pd.DataFrame(pcd_file, columns={"X" ,"Y" ,"Z"})
            self.pcd_data = self.pcd_data[0:self.nrows]
        else:
            raise Exception(f"Unsupported datatype of pcd file.")
        self.kdtree_main = KdTree_class()
        self.kdtree_root_node = self.kdtree_main.insert_points(self.pcd_data, display_output=display_output_flag)

    def euclidean_clustering(self, distance_threshold, cluster_parameters):
        """
        Perform Euclidean clustering
        :param distance_threshold: minimum distance between points in a cluster
        :param cluster_parameters: {min_size:value_1}
        :return: identified clusters after Euclidean Clustering
        """
        clusters_identified = dict()
        cluster_id = 0
        # Set up a boolean list with all elements set to False
        processed_flag = [False]*self.nrows
        # Each row in pcd_data is a pcd point
        for point in self.pcd_data.iterrows():
            index = point[0]
            current_point = (point[1]["X"], point[1]["Y"], point[1]["Z"])
            if not processed_flag[index]:
                base_cluster = set()
                self.find_clusters(current_point, base_cluster, index, distance_threshold, cluster_parameters, processed_flag)
                # If cluster is within the parameter limits
                if base_cluster is not None:
                    if (len(base_cluster) > cluster_parameters["min_size"]):
                        clusters_identified[cluster_id] = base_cluster
                        cluster_id += 1
        return clusters_identified

    def get_point(self, index):
        """
        return point
        :param index: dataframe index
        :return: point as tuple
        """
        return (self.pcd_data.loc[index]["X"],
                self.pcd_data.loc[index]["Y"],
                self.pcd_data.loc[index]["Z"])

    def find_clusters(self, current_point_, base_cluster_, index_, threshold_, cluster_parameters_, processed_flag_):
        """
        search for clusters
        :param current_point_: current point being processed.
        :param base_cluster_: already known clusters.
        :param index_: index of the point.
        :param threshold_:  min distance for cluster identification.
        :param cluster_parameters_: Cluster parameters {min_size}
        :param processed_flag_: list containing status of points which have been identified as clusters.
        """
        # Considering points which have not been processed before, and are within are cluster parameter limits
        if not processed_flag_[index_]:
            processed_flag_[index_] = True
            base_cluster_.add(index_)
            # Returns id listof the points which are near the current point,
            # id and index would be the same because of the unpacking design followed in build_kdtree method.
            nearby_points = self.kdtree_main.search_elements(node=self.kdtree_root_node ,search_point=current_point_ ,
                                                             distance_threshold=threshold_ ,depth=0)
            # This line was added to retain the orginal variable during the process of recursion
            # Using the same variable resulted in changing the contents during iteration
            # which raised an exception.
            nearby_points_ = copy.copy(nearby_points)
            # If points are present search for more clusters
            if len(nearby_points_):
                for index_ in nearby_points_:
                    if not processed_flag_[index_]:
                        # Get the point in (x,y,z) for as required by the method
                        point = self.get_point(index_)
                        # Recursively iterate through all the points
                        self.find_clusters(point, base_cluster_, index_,
                                           threshold_, cluster_parameters_, processed_flag_)

    def visualize_clusters(self, clusters):
        """
        Visualization of point cloud data
        :param clusters: identified clusters
        :return:
        """
        trace_1 = go.Scatter3d(x=self.pcd_data.X,
                             y=self.pcd_data.Y,
                             z=self.pcd_data.Z,
                             mode='markers')
        # extract points from clusters
        cluster_points = pd.DataFrame(columns={"Cluster_id", "X", "Y", "Z"})
        for key in clusters:
            # Get all the indexes related to the cluster id = key
            cluster_point_indexes = clusters[key]
            for point_index in cluster_point_indexes:
                point = self.get_point(point_index)
                cluster_points = cluster_points.append({"Cluster_id": key,
                                                         "X": point[0],
                                                         "Y": point[1],
                                                         "Z": point[2]}, ignore_index=True)
        unique_clusters = []
        for cluster_id in cluster_points.Cluster_id.unique():
            cluster_traces = [go.Scatter3d(x=cluster_points[cluster_points["Cluster_id"] == cluster_id].X ,
                                   y=cluster_points[cluster_points["Cluster_id"] == cluster_id].Y ,
                                   z=cluster_points[cluster_points["Cluster_id"] == cluster_id].Z ,
                                   mode='markers')]
            unique_clusters.extend(cluster_traces)
        data = [trace_1]
        data.extend(unique_clusters)
        fig = go.Figure(data)
        fig.show()


if __name__ == "__main__":
    # APPLICATION = ProcessPointCloud(pcd_file="point_cloud_data_sample.xyz", nrows_value=100, display_output_flag=True)
    APPLICATION = ProcessPointCloud(pcd_file=lidar ,nrows_value=6000, display_output_flag=False)
    clusters = APPLICATION.euclidean_clustering(distance_threshold=5, cluster_parameters={"min_size":2})
    # print(clusters)
    APPLICATION.visualize_clusters(clusters)